In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sheila/

In [2]:
#%%
scenario = "basic"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "basic",
    "nWifi": 2,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None


Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:33933
Please start proper ns-3 simulation script using ./waf --run "..."
34.781616|0|0|2|-nan
Observation space shape: (1, 300)
Action space shape: (1, 1)


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4

agent_count = 2 # Number of agents to train
agents = []    # List to store the agents

for i in range(agent_count):
    config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
    agent = Agent(QNetworkTf, history_length, action_size=7, config=config)
    agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)
    agents.append(agent)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agents, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
logger = teacher.eval(agents,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        tags=tags,
                        parameters=hyperparams)
# agent.save()


# %%





The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





Action space: 7
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Action space: 7


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi/dfa78bdf277746bd8c2d5dd815abb324



...

100%|██████████| 6300/6300 [10:42<00:00,  9.81it/s, curr_speed=37.55 Mbps, mb_sent=4513.34 Mb]


Waiting for simulation script to connect on port: tcp://localhost:39193
Please start proper ns-3 simulation script using ./waf --run "..."


COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi/dfa78bdf277746bd8c2d5dd815abb324
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [179970]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]       : (1583.840576171875, 1603.30517578125)
COMET INFO:     Current throughput [179970]  : (35.7421989440918, 41.326900482177734)
COMET INFO:     Fairness index [179970]      : (2.0, 2.0)
COMET INFO:     Megabytes sent [179970]      : (0.35742199420928955, 4573.637411117554)
COMET INFO:     Observation 0 [179970]       : (0.8183515311529238, 0.862770741350121)
COMET INFO:     Observation 1 [179970]       : (0.018067934457237533, 0.07551361583437353)
COMET INFO:     Per-ep reward [179970]       : (75.4234619140625, 1603.30517578125)
COMET INFO:     Round megabytes sent [179970]: (0.2680659890174866, 0.44677698612213135)
COMET INFO:     Round rew

Sent 4513.34 Mb/s.	Mean speed: 75.22 Mb/s	Episode 15/15 finished

Training finished.


AttributeError: 'list' object has no attribute 'load'